In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as datetime

In [3]:
import kagglehub

path = kagglehub.dataset_download("usdot/flight-delays")

print("Path to dataset files:", path)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/gustavocoradin/.cache/kagglehub/datasets/usdot/flight-delays/versions/1


In [4]:
import pandas as pd
df = pd.read_csv(path + "/flights.csv")

/var/folders/28/kqrw_4t50j7_wd11ljb0mrmh0000gn/T/ipykernel_7516/286890719.py:2: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path + "/flights.csv")


In [5]:
df.columns

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')

Analisando o dataset / data cleaning

In [6]:
print(df.isna().sum() * 100 / len(df))

YEAR                    0.000000
MONTH                   0.000000
DAY                     0.000000
DAY_OF_WEEK             0.000000
AIRLINE                 0.000000
FLIGHT_NUMBER           0.000000
TAIL_NUMBER             0.252978
ORIGIN_AIRPORT          0.000000
DESTINATION_AIRPORT     0.000000
SCHEDULED_DEPARTURE     0.000000
DEPARTURE_TIME          1.480526
DEPARTURE_DELAY         1.480526
TAXI_OUT                1.530259
WHEELS_OFF              1.530259
SCHEDULED_TIME          0.000103
ELAPSED_TIME            1.805629
AIR_TIME                1.805629
DISTANCE                0.000000
WHEELS_ON               1.589822
TAXI_IN                 1.589822
SCHEDULED_ARRIVAL       0.000000
ARRIVAL_TIME            1.589822
ARRIVAL_DELAY           1.805629
DIVERTED                0.000000
CANCELLED               0.000000
CANCELLATION_REASON    98.455357
AIR_SYSTEM_DELAY       81.724960
SECURITY_DELAY         81.724960
AIRLINE_DELAY          81.724960
LATE_AIRCRAFT_DELAY    81.724960
WEATHER_DE

In [7]:
# trocando NaN por 0 já que 0 aqui significa ausência de atraso
delay_reason_cols = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
df[delay_reason_cols] = df[delay_reason_cols].fillna(0)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.drop(['CANCELLATION_REASON', 'FLIGHT_NUMBER', 'CANCELLED'], axis=1, inplace=True, errors='ignore')

df.drop('TAIL_NUMBER', axis=1, inplace=True)

df.drop(['ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'AIRLINE'], axis=1, inplace=True, errors='ignore')
df = df.dropna(subset=['DEPARTURE_TIME','DEPARTURE_DELAY','TAXI_OUT','WHEELS_OFF','SCHEDULED_TIME','ELAPSED_TIME','AIR_TIME','WHEELS_ON','TAXI_IN','ARRIVAL_TIME','ARRIVAL_DELAY'])

In [9]:
print(df.isna().sum() * 100 / len(df))

YEAR                   0.0
MONTH                  0.0
DAY                    0.0
DAY_OF_WEEK            0.0
SCHEDULED_DEPARTURE    0.0
DEPARTURE_TIME         0.0
DEPARTURE_DELAY        0.0
TAXI_OUT               0.0
WHEELS_OFF             0.0
SCHEDULED_TIME         0.0
ELAPSED_TIME           0.0
AIR_TIME               0.0
DISTANCE               0.0
WHEELS_ON              0.0
TAXI_IN                0.0
SCHEDULED_ARRIVAL      0.0
ARRIVAL_TIME           0.0
ARRIVAL_DELAY          0.0
DIVERTED               0.0
AIR_SYSTEM_DELAY       0.0
SECURITY_DELAY         0.0
AIRLINE_DELAY          0.0
LATE_AIRCRAFT_DELAY    0.0
WEATHER_DELAY          0.0
dtype: float64


In [ ]:

df['DELAYED'] = df['ARRIVAL_DELAY'].apply(lambda x: 1 if x > 15 else 0)

In [11]:
delay = df[df['DELAYED'] == 1]
on_time = df[df['DELAYED'] == 0]

delay_count = delay.shape[0]
on_time_count = on_time.shape[0]

total = delay_count + on_time_count
delay_percentage = (delay_count / total) * 100
on_time_percentage = (on_time_count / total) * 100

print(f"Não atrasados: {on_time_count:,}".replace(",", "."))
print(f"Atrasados: {delay_count:,}".replace(",", "."))
print(f"Percentual de voos não atrasados: {on_time_percentage:.2f}%")
print(f"Percentual de voos atrasados: {delay_percentage:.2f}%")


Não atrasados: 4.690.510
Atrasados: 1.023.498
Percentual de voos não atrasados: 82.09%
Percentual de voos atrasados: 17.91%


In [12]:
from imblearn.under_sampling import RandomUnderSampler

X = df.drop(columns=['DELAYED'])
y = df['DELAYED']

rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X, y)


df_undersampled = pd.concat([X_resampled, y_resampled], axis=1)

on_time_count = (df_undersampled['DELAYED'] == 0).sum()
delay_count = (df_undersampled['DELAYED'] == 1).sum()
total = on_time_count + delay_count


on_time_percentage = (on_time_count / total) * 100
delay_percentage = (delay_count / total) * 100

print(f"Não atrasados: {on_time_count:,}".replace(",", "."))
print(f"Atrasados: {delay_count:,}".replace(",", "."))
print(f"Percentual de voos não atrasados: {on_time_percentage:.2f}%")
print(f"Percentual de voos atrasados: {delay_percentage:.2f}%")

Não atrasados: 1.023.498
Atrasados: 1.023.498
Percentual de voos não atrasados: 50.00%
Percentual de voos atrasados: 50.00%


In [14]:
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,...,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DELAYED
0,2015,1,1,4,5,2354.0,-11.0,21.0,15.0,205.0,...,430,408.0,-22.0,0,0.0,0.0,0.0,0.0,0.0,0
1,2015,1,1,4,10,2.0,-8.0,12.0,14.0,280.0,...,750,741.0,-9.0,0,0.0,0.0,0.0,0.0,0.0,0
2,2015,1,1,4,20,18.0,-2.0,16.0,34.0,286.0,...,806,811.0,5.0,0,0.0,0.0,0.0,0.0,0.0,0
3,2015,1,1,4,20,15.0,-5.0,15.0,30.0,285.0,...,805,756.0,-9.0,0,0.0,0.0,0.0,0.0,0.0,0
4,2015,1,1,4,25,24.0,-1.0,11.0,35.0,235.0,...,320,259.0,-21.0,0,0.0,0.0,0.0,0.0,0.0,0


In [15]:
pd.set_option('display.max_columns', None)
df.head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DELAYED
0,2015,1,1,4,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0.0,0.0,0.0,0.0,0.0,0
1,2015,1,1,4,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0.0,0.0,0.0,0.0,0.0,0
2,2015,1,1,4,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0.0,0.0,0.0,0.0,0.0,0
3,2015,1,1,4,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0.0,0.0,0.0,0.0,0.0,0
4,2015,1,1,4,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0.0,0.0,0.0,0.0,0.0,0


In [16]:
airlines_dict = {
    'UA': 'United Air Lines Inc.',
    'AA': 'American Airlines Inc.',
    'US': 'US Airways Inc.',
    'F9': 'Frontier Airlines Inc.',
    'B6': 'JetBlue Airways',
    'OO': 'Skywest Airlines Inc.',
    'AS': 'Alaska Airlines Inc.',
    'NK': 'Spirit Air Lines',
    'WN': 'Southwest Airlines Co.',
    'DL': 'Delta Air Lines Inc.',
    'EV': 'Atlantic Southeast Airlines',
    'HA': 'Hawaiian Airlines Inc.',
    'MQ': 'American Eagle Airlines Inc.',
    'VX': 'Virgin America'
}


In [17]:
pd.set_option('display.max_columns', None)
df.head(5)

,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,DELAYED
0,2015,1,1,4,5,2354.0,-11.0,21.0,15.0,205.0,194.0,169.0,1448,404.0,4.0,430,408.0,-22.0,0,0.0,0.0,0.0,0.0,0.0,0
1,2015,1,1,4,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0.0,0.0,0.0,0.0,0.0,0
2,2015,1,1,4,20,18.0,-2.0,16.0,34.0,286.0,293.0,266.0,2296,800.0,11.0,806,811.0,5.0,0,0.0,0.0,0.0,0.0,0.0,0
3,2015,1,1,4,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0.0,0.0,0.0,0.0,0.0,0
4,2015,1,1,4,25,24.0,-1.0,11.0,35.0,235.0,215.0,199.0,1448,254.0,5.0,320,259.0,-21.0,0,0.0,0.0,0.0,0.0,0.0,0


# IMPLEMENTAÇÃO KNN

In [ ]:

df_delayed = df[df['DELAYED'] == 1]
df_on_time = df[df['DELAYED'] == 0]

sample_size_per_class = 50000

df_delayed_sample = df_delayed.sample(n=sample_size_per_class, random_state=42)
df_on_time_sample = df_on_time.sample(n=sample_size_per_class, random_state=42)

df_sample = pd.concat([df_delayed_sample, df_on_time_sample])

df_sample = df_sample.sample(frac=1, random_state=42).reset_index(drop=True)

print("Shape do DataFrame original:", df.shape)
print("Shape do DataFrame amostrado:", df_sample.shape)
print("\Distribuição da classe 'DELAYED' no DataFrame amostrado:")
print(df_sample['DELAYED'].value_counts())

Shape do DataFrame original: (5714008, 25)
Shape do DataFrame amostrado: (100000, 25)
\Distribuição da classe 'DELAYED' no DataFrame amostrado:
DELAYED
0    50000
1    50000
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split

X_sample = df_sample.drop(columns=['DELAYED'])
y_sample = df_sample['DELAYED']

X_train, X_test, y_train, y_test = train_test_split(
    X_sample, y_sample,
    test_size=0.3,
    stratify=y_sample,
    random_state=42
)


In [20]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [21]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(X_train_scaled, y_train)


KNeighborsClassifier(n_neighbors=15)

In [22]:
y_pred = knn.predict(X_test_scaled)


In [23]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

tpr = recall  # mesmo que tp / (tp + fn)
tnr = tn / (tn + fp)

cm = confusion_matrix(y_test, y_pred)

print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall (TPR):", recall)
print("F1-score:", f1)
print("True Negative Rate (TNR):", tnr)
print("Matriz de Confusão:")
print(cm)


Acurácia: 0.9453
Precisão: 0.9880891487029595
Recall (TPR): 0.9014666666666666
F1-score: 0.9427924002091685
True Negative Rate (TNR): 0.9891333333333333
Matriz de Confusão:
[[14837   163]
 [ 1478 13522]]


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_train_pred = knn.predict(X_train_scaled)

accuracy_train = accuracy_score(y_train, y_train_pred)
precision_train = precision_score(y_train, y_train_pred)
recall_train = recall_score(y_train, y_train_pred)
f1_train = f1_score(y_train, y_train_pred)
tn, fp, fn, tp = confusion_matrix(y_train, y_train_pred).ravel()
tpr_train = recall_train  # tp / (tp + fn)
tnr_train = tn / (tn + fp)

print("### Métricas no TREINO ###")
print("Acurácia:", accuracy_train)
print("Precisão:", precision_train)
print("Recall (TPR):", recall_train)
print("F1-score:", f1_train)
print("True Negative Rate (TNR):", tnr_train)

print("\n### Métricas no TESTE ###")
print("Acurácia:", accuracy)
print("Precisão:", precision)
print("Recall (TPR):", recall)
print("F1-score:", f1)
print("True Negative Rate (TNR):", tnr)


### Métricas no TREINO ###
Acurácia: 0.9544
Precisão: 0.9910460664443621
Recall (TPR): 0.9170857142857143
F1-score: 0.9526325161749867
True Negative Rate (TNR): 0.9917142857142857

### Métricas no TESTE ###
Acurácia: 0.9453
Precisão: 0.9880891487029595
Recall (TPR): 0.9014666666666666
F1-score: 0.9427924002091685
True Negative Rate (TNR): 0.9891333333333333


In [ ]:
print("\n" + "="*50)
print("VALIDAÇÃO CRUZADA 5-FOLDS")
print("="*50)

from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=15))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(pipeline, X_sample, y_sample, cv=cv, scoring=scoring, return_train_score=True)

print(f"Acurácia - Treino: {cv_results['train_accuracy'].mean():.4f} ± {cv_results['train_accuracy'].std():.4f}")
print(f"Acurácia - Validação: {cv_results['test_accuracy'].mean():.4f} ± {cv_results['test_accuracy'].std():.4f}")
print(f"Precisão - Treino: {cv_results['train_precision'].mean():.4f} ± {cv_results['train_precision'].std():.4f}")
print(f"Precisão - Validação: {cv_results['test_precision'].mean():.4f} ± {cv_results['test_precision'].std():.4f}")
print(f"Recall - Treino: {cv_results['train_recall'].mean():.4f} ± {cv_results['train_recall'].std():.4f}")
print(f"Recall - Validação: {cv_results['test_recall'].mean():.4f} ± {cv_results['test_recall'].std():.4f}")
print(f"F1-score - Treino: {cv_results['train_f1'].mean():.4f} ± {cv_results['train_f1'].std():.4f}")
print(f"F1-score - Validação: {cv_results['test_f1'].mean():.4f} ± {cv_results['test_f1'].std():.4f}")

acc_diff = cv_results['train_accuracy'].mean() - cv_results['test_accuracy'].mean()
if acc_diff > 0.05:
    print(f"\n⚠️  POSSÍVEL OVERFITTING DETECTADO!")
    print(f"Diferença de acurácia treino-validação: {acc_diff:.4f}")
else:
    print(f"\n✅ Modelo parece estar generalizando bem.")
    print(f"Diferença de acurácia treino-validação: {acc_diff:.4f}")



VALIDAÇÃO CRUZADA 5-FOLDS
Acurácia - Treino: 0.9564 ± 0.0004
Acurácia - Validação: 0.9476 ± 0.0017
Precisão - Treino: 0.9911 ± 0.0003
Precisão - Validação: 0.9895 ± 0.0014
Recall - Treino: 0.9210 ± 0.0007
Recall - Validação: 0.9048 ± 0.0026
F1-score - Treino: 0.9548 ± 0.0004
F1-score - Validação: 0.9453 ± 0.0019

✅ Modelo parece estar generalizando bem.
Diferença de acurácia treino-validação: 0.0088


In [26]:
import time

start_time_train = time.time()
knn.fit(X_train_scaled, y_train)
end_time_train = time.time()

training_time = end_time_train - start_time_train
print(f"Training Time: {training_time:.4f} seconds")


Training Time: 0.0606 seconds


**Reasoning**:
Measure the prediction time of the KNN model and then display both training and prediction times.



In [27]:
pip install memory-profiler


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install psutil memory-profiler



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 1.6005 seconds


In [30]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 1.7117 seconds


In [31]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 1.4549 seconds


In [32]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 1.5390 seconds


In [33]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 1.4483 seconds


In [34]:
start_time_pred = time.time()
y_pred = knn.predict(X_test_scaled)
end_time_pred = time.time()

prediction_time = end_time_pred - start_time_pred
print(f"Prediction Time: {prediction_time:.4f} seconds")


Prediction Time: 1.4088 seconds


In [35]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

knn_cv = KNeighborsClassifier(n_neighbors=15)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

cv_results = cross_validate(knn_cv, X_sample, y_sample, cv=cv, scoring=scoring)

print("=== Validação Cruzada 5-Folds ===")
print(f"Acurácia média: {cv_results['test_accuracy'].mean():.4f}")
print(f"Precisão média: {cv_results['test_precision'].mean():.4f}")
print(f"Recall médio: {cv_results['test_recall'].mean():.4f}")
print(f"F1-score médio: {cv_results['test_f1'].mean():.4f}")


=== Validação Cruzada 5-Folds ===
Acurácia média: 0.9150
Precisão média: 0.9633
Recall médio: 0.8629
F1-score médio: 0.9103


In [ ]:
import time
import psutil
from memory_profiler import memory_usage
from sklearn.neighbors import KNeighborsClassifier

process = psutil.Process()

knn = KNeighborsClassifier(n_neighbors=5)

def train_model():
    knn.fit(X_train_scaled, y_train)

start_time_train = time.time()
mem_usage_train = memory_usage(train_model)
end_time_train = time.time()
training_time = end_time_train - start_time_train
train_ips = len(X_train_scaled) / training_time

def predict_model():
    global y_pred
    y_pred = knn.predict(X_test_scaled)

cpu_percent_before = process.cpu_percent(interval=None)
start_time_pred = time.time()
mem_usage_pred = memory_usage(predict_model)
end_time_pred = time.time()
cpu_percent_after = process.cpu_percent(interval=None)

prediction_time = end_time_pred - start_time_pred
pred_ips = len(X_test_scaled) / prediction_time

print("=== DESEMPENHO COMPUTACIONAL ===")
print(f"🕒 Tempo de Treinamento: {training_time:.4f} s")
print(f"🕒 Tempo de Predição: {prediction_time:.4f} s")
print(f"📈 Memória (Treinamento): {max(mem_usage_train):.2f} MB")
print(f"📈 Memória (Predição): {max(mem_usage_pred):.2f} MB")
print(f"⚙️ CPU usada na predição: {cpu_percent_after:.2f}%")
print(f"📊 Instâncias por segundo (treinamento): {train_ips:.2f}")
print(f"📊 Instâncias por segundo (predição): {pred_ips:.2f}")


=== DESEMPENHO COMPUTACIONAL ===
🕒 Tempo de Treinamento: 0.8252 s
🕒 Tempo de Predição: 1.8882 s
📈 Memória (Treinamento): 146.69 MB
📈 Memória (Predição): 156.52 MB
⚙️ CPU usada na predição: 454.80%
📊 Instâncias por segundo (treinamento): 84831.61
📊 Instâncias por segundo (predição): 15888.17
